In [2]:
from appgeopy import *
from my_packages import *
from scipy.stats import pearsonr
from sklearn.preprocessing import StandardScaler  # or MinMaxScaler

# Input Files

## correlation plot

## scatter plot

## timeseries plot

# GTWR Results

## scatter plot - Observations & Modeled

In [3]:
files = glob(os.path.join("gtwr_run*/", "gtwr*Layer_*results.csv"))
files

['gtwr_run_output_Layer_1\\gtwr_Layer_1_regression_kernel-tricube_lambda-0d03_bw-25_results.csv',
 'gtwr_run_output_Layer_2\\gtwr_Layer_2_regression_kernel-tricube_lambda-0d002_bw-18_results.csv',
 'gtwr_run_output_Layer_3\\gtwr_Layer_3_regression_kernel-tricube_lambda-0d07_bw-17_results.csv',
 'gtwr_run_output_Layer_4\\gtwr_Layer_4_regression_kernel-tricube_lambda-0d1_bw-18_results.csv']

In [6]:
select_file = files[0]
df = pd.read_csv(select_file)
df.head(3)

,input_STATION,input_time,input_Layer_1,input_CUMDISP,X_TWD97,Y_TWD97,Time_value,Intercept,CUMDISP,predicted_value,prediction_error,absolute_error
0,BEICHEN,2016-05-01,-1.0,-3.552222,178859.958807,2.608229e+06,1,-0.719650,0.310292,-1.821877,0.821877,0.821877
1,BEICHEN,2016-06-01,-1.0,-10.291302,178859.958807,2.608229e+06,2,-1.583061,0.071473,-2.318606,1.318606,1.318606
2,BEICHEN,2016-07-01,-1.0,-15.785017,178859.958807,2.608229e+06,3,-0.450648,0.105084,-2.109397,1.109397,1.109397


In [12]:
FIGURE_SIZE = (16.5, 11.7)  # A3 landscape
OUTPUT_DPI = 300

# select_file = files[0]
for select_file in files[:]:

    curent_layer = "Layer_" + os.path.basename(select_file).split("_")[2]

    # ==============================================================================
    # LOAD AND PREPARE DATA
    # ==============================================================================
    df = pd.read_csv(select_file)
    df = df.query("Time_value<=67")

    stations = sorted(df["input_STATION"].unique())
    n_stations = len(stations)

    # Calculate grid dimensions (aim for roughly square layout)
    n_cols = int(np.ceil(np.sqrt(n_stations)))
    n_rows = int(np.ceil(n_stations / n_cols))

    # ==============================================================================
    # CREATE SUBPLOTS
    # ==============================================================================
    fig, axes = plt.subplots(
        n_rows, n_cols, figsize=FIGURE_SIZE, sharex=False, sharey=False
    )
    axes = axes.flatten()  # Convert to 1D array for easy indexing

    # Main title
    fig.suptitle(
        f"{curent_layer}",
        fontsize=24,
        fontweight="bold",
        y=0.985,
    )

    # ==============================================================================
    # PLOT EACH STATION
    # ==============================================================================
    for idx, station in enumerate(stations):
        ax = axes[idx]

        # Get station data
        station_data = df.query("input_STATION==@station").sort_values(
            by="Time_value"
        )
        x = station_data[f"input_{curent_layer}"].values
        y = station_data["predicted_value"].values

        # Calculate correlation
        if len(x) >= 3:
            r, p = pearsonr(x, y)

            # Scatter plot
            ax.scatter(x, y, alpha=0.6, s=30, edgecolors="black", linewidth=0.5)

            # Best fit line
            z = np.polyfit(x, y, 1)
            p_fit = np.poly1d(z)
            x_line = np.linspace(x.min(), x.max(), 100)
            ax.plot(x_line, p_fit(x_line), "r-", linewidth=2, alpha=0.7)

            # Statistics text
            sig_marker = "*" if p < 0.05 else ""
            stats_text = f"r = {r:.3f}{sig_marker}\nn = {len(x)}"
            ax.text(
                0.05,
                0.95,
                stats_text,
                transform=ax.transAxes,
                va="top",
                fontsize=9,
                family="monospace",
                bbox=dict(boxstyle="round", facecolor="white", alpha=0.8),
            )

        # Styling
        ax.set_title(station, fontsize=11, fontweight="bold")
        # ax.set_xlabel("Obs.", fontsize=9)
        # ax.set_ylabel("Sim.", fontsize=9)
        ax.grid(True, alpha=0.3, linestyle="--")
        ax.tick_params(labelsize=8)

    # Hide unused subplots
    for idx in range(n_stations, len(axes)):
        axes[idx].axis("off")

    # Add single shared axis labels
    fig.text(
        0.5, 0.02, "Observed (mm)", ha="center", fontsize=14, fontweight="bold"
    )
    fig.text(
        0.02,
        0.5,
        "Modeled (mm)",
        va="center",
        rotation="vertical",
        fontsize=14,
        fontweight="bold",
    )

    # ==============================================================================
    # SAVE AND DISPLAY
    # ==============================================================================

    fig.tight_layout(rect=[0.03, 0.03, 1, 0.98])  # Leave margins for labels
    visualize.save_figure(
        fig=fig,
        savepath=os.path.join(f"scatterplot_obs_sim_{curent_layer}.png"),
    )
    plt.close()